In [60]:
# coding:utf-8
import arrow
import qgrid
import seaborn as sns
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
%matplotlib inline

sns.set(style='whitegrid')

In [61]:
from pyhanlp import *

In [62]:
print(HanLP.segment('李泽钜接手后的屈臣氏：前有丝芙兰后有KOL 腾讯阿里争买股份 谁才是最大的赢家'))

[李泽钜/nr, 接手/v, 后/f, 的/ude1, 屈臣氏/nr, ：/w, 前/f, 有/vyou, 丝/n, 芙/ng, 兰后/j, 有/vyou, KOL/nx,  /w, 腾讯/ntc, 阿里/nt, 争/v, 买/v, 股份/n,  /w, 谁/ry, 才/d, 是/vshi, 最大/gm, 的/ude1, 赢家/n]


In [63]:
my_hk_news_all = pd.read_csv("../tmp_data/my_hk_news_all.csv",encoding='utf-8',index_col = False, delimiter=',' )
my_hk_news_all.head()

,title,url,ctime,label_type,label_property,label_article,code,company,industry
0,香港老牌发展商合和实业今日除牌 结束近47年上市地位,https://news.10jqka.com.cn/m611192568/,1556849040,新闻,中性,"50019340_除牌,50023242_合和实业",HK0001,长和,综合
1,赫斯基首季盈利增32%胜预期,https://news.10jqka.com.cn/m611120552/,1556349944,新闻,中性,"50000652_项目投资,50006059_原油价格",HK0001,长和,综合
2,17年之后 李嘉诚长和系公司再度申请在香港主板上市,https://m.21jingji.com/article/20190416/herald...,1555418857,新闻,中性,"50004955_李嘉诚,50015283_主板",HK0001,长和,综合
3,李泽钜接手后的屈臣氏：前有丝芙兰后有KOL 腾讯阿里争买股份 谁才是最大的赢家,https://m.jiemian.com/article/3002332.html,1554091843,新闻,中性,"50001089_腾讯,50004918_阿里巴巴",HK0001,长和,综合
4,东兴证券2019年4月十大金股,https://mp.weixin.qq.com/s?__biz=MzA3NTA2NTUwO...,1553913617,新闻,中性,"50004118_东兴证券,50009633_风险警示",HK0001,长和,综合


In [64]:
my_hk_news_all = my_hk_news_all.drop(['ctime','label_type','label_article','industry','url','label_property','code','company'],axis=1)
my_hk_news_all.head()

,title
0,香港老牌发展商合和实业今日除牌 结束近47年上市地位
1,赫斯基首季盈利增32%胜预期
2,17年之后 李嘉诚长和系公司再度申请在香港主板上市
3,李泽钜接手后的屈臣氏：前有丝芙兰后有KOL 腾讯阿里争买股份 谁才是最大的赢家
4,东兴证券2019年4月十大金股


In [65]:
def cut_word(sentence):
    term_nature = []
    
    
    termlist = HanLP.segment(sentence.strip())
#print(len(sentence.strip()))
    
    for term in termlist:
        str_term = str(term)

        str_arr = str_term.split('/')
        if str_arr[1] == 'company' or str_arr[1] == 'nt' or  str_arr[1] == 'ntc' :
            len_word = len(str_arr[0])
            for i in range(0,len_word):
                if i==0: 
                    term_nature.append('B-ORG')
                else:
                    term_nature.append('I-ORG')
        else:
            len_word = len(str_arr[0])
            for i in range(0,len_word):
                term_nature.append('O')
    if len(term_nature) == len(sentence.strip()):
        return " ".join(term_nature)
    else :
        return "".join(term_nature)

my_hk_news_all['noun'] = my_hk_news_all['title'].apply(cut_word)
my_hk_news_all.head()

,title,noun
0,香港老牌发展商合和实业今日除牌 结束近47年上市地位,O O O O O O O B-ORG I-ORG I-ORG I-ORG O O O O ...
1,赫斯基首季盈利增32%胜预期,O O O O O O O O O O O O O O
2,17年之后 李嘉诚长和系公司再度申请在香港主板上市,O O O O O O O O O B-ORG I-ORG O O O O O O O O ...
3,李泽钜接手后的屈臣氏：前有丝芙兰后有KOL 腾讯阿里争买股份 谁才是最大的赢家,O O O O O O O O O O O O O O O O O O O O O O B-...
4,东兴证券2019年4月十大金股,B-ORG I-ORG I-ORG I-ORG O O O O O O O O O O O


In [66]:
def cut_word_tmp(x):
    word_seg = []
    if len(x['title'].strip()) == len(x['noun'].split()):
        for word in x['title']:
            word_seg.append(word)
        return "".join(word_seg)
        
    else:
        return ""

            

my_hk_news_all['word_seg'] = my_hk_news_all.apply(cut_word_tmp,axis=1)
my_hk_news_all.head()

,title,noun,word_seg
0,香港老牌发展商合和实业今日除牌 结束近47年上市地位,O O O O O O O B-ORG I-ORG I-ORG I-ORG O O O O ...,香港老牌发展商合和实业今日除牌 结束近47年上市地位
1,赫斯基首季盈利增32%胜预期,O O O O O O O O O O O O O O,赫斯基首季盈利增32%胜预期
2,17年之后 李嘉诚长和系公司再度申请在香港主板上市,O O O O O O O O O B-ORG I-ORG O O O O O O O O ...,17年之后 李嘉诚长和系公司再度申请在香港主板上市
3,李泽钜接手后的屈臣氏：前有丝芙兰后有KOL 腾讯阿里争买股份 谁才是最大的赢家,O O O O O O O O O O O O O O O O O O O O O O B-...,李泽钜接手后的屈臣氏：前有丝芙兰后有KOL 腾讯阿里争买股份 谁才是最大的赢家
4,东兴证券2019年4月十大金股,B-ORG I-ORG I-ORG I-ORG O O O O O O O O O O O,东兴证券2019年4月十大金股


In [67]:
my_hk_news_all = my_hk_news_all[my_hk_news_all['word_seg'] != '']

In [71]:
my_a_news_all = pd.read_csv("../a_stock_news/result.csv",encoding='utf-8',index_col = False, delimiter=',' )
my_a_news_all.head()

,title,url,ctime,label_type,label_property,label_article,code,company,industry
0,国家能源局：优先建设平价上网风电、光伏发电项目,https://news.10jqka.com.cn/m610764673/,1554892969,新闻,中性,50011009_光伏概念,600207,安彩高科,燃气
1,“性价比”：光伏企业走向平价关键词,https://news.10jqka.com.cn/m610761460/,1554890486,新闻,中性,50010720_光伏概念,600207,安彩高科,燃气
2,国家能源局：先行确定一批2019年度可开工建设的平价上网风电、光伏发电项目,https://news.10jqka.com.cn/m610759719/,1554887065,新闻,中性,50011009_光伏概念,600207,安彩高科,燃气
3,安彩高科2018年营收增稳增 打赢经营和机制建设攻坚战,https://news.10jqka.com.cn/m610424435/,1553263893,新闻,中性,"50003432_安彩高科,50000428_年度报告",600207,安彩高科,燃气
4,公告解读：安彩高科去年全年亏损3.33亿,https://news.10jqka.com.cn/m610418116/,1553250734,新闻,NaN,50000272_定期报告,600207,安彩高科,燃气


In [72]:
my_a_news_all = my_a_news_all.drop(['ctime','label_type','label_article','industry','url','label_property','code','company'],axis=1)
my_a_news_all.head()

,title
0,国家能源局：优先建设平价上网风电、光伏发电项目
1,“性价比”：光伏企业走向平价关键词
2,国家能源局：先行确定一批2019年度可开工建设的平价上网风电、光伏发电项目
3,安彩高科2018年营收增稳增 打赢经营和机制建设攻坚战
4,公告解读：安彩高科去年全年亏损3.33亿


In [73]:
my_a_news_all['noun'] = my_a_news_all['title'].apply(cut_word)
my_a_news_all.head()

,title,noun
0,国家能源局：优先建设平价上网风电、光伏发电项目,O O O O O O O O O O O O O O O O O O O O O O O
1,“性价比”：光伏企业走向平价关键词,O O O O O O O O O O O O O O O O O
2,国家能源局：先行确定一批2019年度可开工建设的平价上网风电、光伏发电项目,O O O O O O O O O O O O O O O O O O O O O O O ...
3,安彩高科2018年营收增稳增 打赢经营和机制建设攻坚战,O O O O O O O O O O O O O O O O O O O O O O O ...
4,公告解读：安彩高科去年全年亏损3.33亿,O O O O O O O O O O O O O O O O O O O O


In [74]:
my_a_news_all['word_seg'] = my_a_news_all.apply(cut_word_tmp,axis=1)
my_a_news_all.head()

,title,noun,word_seg
0,国家能源局：优先建设平价上网风电、光伏发电项目,O O O O O O O O O O O O O O O O O O O O O O O,国家能源局：优先建设平价上网风电、光伏发电项目
1,“性价比”：光伏企业走向平价关键词,O O O O O O O O O O O O O O O O O,“性价比”：光伏企业走向平价关键词
2,国家能源局：先行确定一批2019年度可开工建设的平价上网风电、光伏发电项目,O O O O O O O O O O O O O O O O O O O O O O O ...,国家能源局：先行确定一批2019年度可开工建设的平价上网风电、光伏发电项目
3,安彩高科2018年营收增稳增 打赢经营和机制建设攻坚战,O O O O O O O O O O O O O O O O O O O O O O O ...,安彩高科2018年营收增稳增 打赢经营和机制建设攻坚战
4,公告解读：安彩高科去年全年亏损3.33亿,O O O O O O O O O O O O O O O O O O O O,公告解读：安彩高科去年全年亏损3.33亿


In [77]:
my_a_news_all = my_a_news_all[my_a_news_all['word_seg'] != '']

In [76]:
my_hk_news_all.shape

(65632, 3)

In [78]:
my_hk_news_all = my_hk_news_all.append(my_a_news_all)
my_hk_news_all.shape

(304740, 3)

In [80]:
my_hk_news_all = my_hk_news_all.drop_duplicates(subset='title', keep='last')
my_hk_news_all.shape

(171996, 3)

In [87]:
from sklearn.model_selection import train_test_split
train, test= train_test_split(my_hk_news_all, shuffle=True,test_size=0.1)

In [94]:
test.shape

(17200, 3)

In [96]:
train[['noun','word_seg']].to_csv("stock_title_train.csv",index=None, sep=',' )

In [97]:
test[['noun','word_seg']].to_csv("stock_title_test.csv",index=None, sep=',' )